In [ ]:
from socialvec.socialvec import SocialVec
import pandas as pd
import swifter
import ast
from scipy.spatial.distance import cosine
from tqdm import tqdm
tqdm.pandas()

import os

In [ ]:
import ast
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import gdown

In [ ]:
# Read all in order to get the number of followers per account (in our database)

In [ ]:


# Google Drive file ID for all_usa_users.pkl
file_id = '1flhTAfxODFHze1lmmgG9CB7LHnsFIhUd'

# Construct the direct download URL
url = f"https://drive.google.com/uc?id={file_id}"

# Download the file using gdown
output = "all_usa_users.pkl"


if not os.path.exists(output):
    gdown.download(url, output, quiet=False)

df_all = pd.read_pickle(output, compression='gzip')    

In [ ]:
df_all = df_all[df_all['friends_ids'].notna()]

In [ ]:
def parse_friends_ids(row):
    friends_ids_str = row.get("friends_ids")
    if friends_ids_str is None or not isinstance(friends_ids_str, str):
        return []  # Return an empty list if the value is None or not a string
    try:
        friends_ids_str = friends_ids_str.strip("'")  # Remove the enclosing quotes
        friends_ids = ast.literal_eval(friends_ids_str)  # Safely evaluate the string
        if friends_ids and isinstance(friends_ids, list):
            return friends_ids[0]  # Extract the inner list if it exists
    except (SyntaxError, ValueError, TypeError):  # Catch potential parsing errors
        return []  # Return an empty list if parsing fails
    return []  # Return an empty list if the structure is unexpected

In [ ]:
# Parse the string to a list
def parse_friends_ids(row):
    friends_ids_str = row["friends_ids"].strip("'")
    friends_ids = ast.literal_eval(friends_ids_str)  # Safely evaluate the string
    if friends_ids:
        return friends_ids[0]  # Extract the inner list if it exists
    return []

In [ ]:
# Create dataframe with count of followers per account

df_all["friends_ids_parsed"] = df_all.progress_apply(parse_friends_ids, axis=1)
exploded_df = df_all.explode("friends_ids_parsed")
df_count = exploded_df.groupby(by=['friends_ids_parsed']).count()


# Initialize SV

In [ ]:
sv = SocialVec()

In [ ]:
file_id = '1hAK8XNlveGqGJB5VtLax9g6NiaHKsMh9'
url = f"https://drive.google.com/uc?id={file_id}"
output = "../data/Labeled_data.csv"

if not os.path.exists(output):
    gdown.download(url, output, quiet=False)

wiki_df = pd.read_csv(output)

In [ ]:
popular_df = pd.read_pickle("https://www.dropbox.com/scl/fi/mmzt5rd1o11a85fxuzczs/users_with_over_200_DETAILS.pkl?rlkey=q4wy51zpl5s4rqiagskygu2mg&st=0m95xyae&dl=1")

## Tidy up the data and merge

In [ ]:
popular_df.drop(popular_df.columns[popular_df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
wiki_df.drop(columns=[col for col in wiki_df.columns if col in popular_df.columns], inplace=True)
df = pd.merge(wiki_df, popular_df, left_on='twitter_screen_name', right_on='screen_name')
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df_count = df_count[['friends_ids']]
df_count.reset_index(inplace=True)
df_count.rename(columns= {'friends_ids_parsed':'twitter_user_id'}, inplace=True)
df = pd.merge(df, df_count, on='twitter_user_id')

# Create Popular Accounts Excel based on specific topics

## Remove items with no dbpedia types

In [ ]:
df['dbpedia_types_tokens_hier'] = df['dbpedia_types_tokens_hier'].progress_apply(lambda x: eval(x))

In [ ]:
df['token_len'] = df['dbpedia_types_tokens_hier'].apply(lambda x: len(x))

print(df.shape[0])
df = df[df['token_len']>0]
print(df.shape[0])

In [ ]:
df['category'] = df['dbpedia_types_tokens_hier'].progress_apply(lambda x: max(x, key=x.get))

In [ ]:
df['dbpedia_types_tokens'] = df['dbpedia_types_tokens'].progress_apply(lambda x: eval(x))

In [ ]:
df.drop_duplicates(subset=['wikidata_label','twitter_name'], keep='first', inplace=True)

## Create excel with the most popular accounts per requested topic 
* Now create the file based on the categories plus some additional according to the description
* (based on wikidata keyword/ label and popularity in our US based dataset)

In [ ]:
category_counts = df["dbpedia_types_tokens"].explode().value_counts().to_frame().reset_index()

In [ ]:
category_counts.to_excel('Wikidata_category_counts.xlsx')

In [ ]:
if not os.path.exists(output):
    gdown.download(url, output, quiet=False)

In [ ]:
topics = [
    "MusicalArtist", "Actor", "Politician", "TelevisionShow", "TelevisionStation",
    "Comedian", "SportsTeam", "Film", "TelevisionHost", "Athlete",
    "Journalist", "Band", "Magazine", "Scientist", "Restaurant",
    "SportsTeam", "Event", "BusinessPerson", "Economist", "Youtuber",
    "Device", "Comic", "Sport"
]

In [ ]:
category_counts.iloc[200:300]['dbpedia_types_tokens'].values

In [ ]:
description_keywords = [
    'car ','news','brand','company'
]

In [ ]:
file_path = '../data/popular_accounts.xlsx'

# Save DataFrames to different tabs
with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
    for topic in topics:
        filtered_df = df[df["dbpedia_types_tokens"].apply(lambda tokens: topic in tokens)]
        #filtered_df = df[df['category']==topic]
        filtered_df = filtered_df.sort_values(by='friends_ids', ascending=False).head(100)
        filtered_df.to_excel(writer, sheet_name=topic, index=False)
        
    filtered_df = df[(~df['wikidata_desc'].isna())&
       ((df['wikidata_desc'].str.contains('fashion'))|(df['wikidata_desc'].str.contains('cloth')))&
       (df['wikidata_desc'].str.contains('company'))]
    filtered_df = filtered_df.sort_values(by='friends_ids', ascending=False).head(100)
    filtered_df.to_excel(writer, sheet_name='fashion', index=False)

    for keyword in description_keywords:
        filtered_df = df[(~df['wikidata_desc'].isna())&
           (df['wikidata_desc'].str.contains(keyword))]    
        filtered_df = filtered_df.sort_values(by='friends_ids', ascending=False).head(100)
        filtered_df.to_excel(writer, sheet_name=keyword, index=False)

    filtered_df = df[(~df['wikidata_desc'].isna())&
       ((df['wikidata_desc'].str.contains('park'))|(df['wikidata_desc'].str.contains('attraction')))]
    filtered_df = filtered_df.sort_values(by='friends_ids', ascending=False).head(100)
    filtered_df.to_excel(writer, sheet_name='attractions', index=False)


# Recommendation Similarities

Implementation of similarity according to SocialVec

In [ ]:
def check_if_same_type(user_id,etype):
    condition = (df['twitter_user_id'] == user_id) & \
            df['dbpedia_types_tokens'].apply(lambda x: etype in x)
    return condition.any()

def new_get_similar(input: str,etype: str,topn: int = 10):
        """
        This function returns the topn similar entities for a given entity

        Parameters
        ----------
        input : twitter user id or username
        by : 'userid', 'username' or vector default is username
        topn : requested numner of similar entities

        Returns
        -------
        Pandas dataframe with the top n similar entities details

        """

        if isinstance(input, int) or (isinstance(input, str) and input.isdigit()):
            input = sv.validate_userid(input)
        elif isinstance(input, str):
            input = sv.validate_username(input)
        #else input is a 'vector'
        #etype = sv.entities[sv.entities['twitter_id']==(input)]['final_type_merged'].iloc[0]
        sim = sv.sv.wv.most_similar(input, topn=500)
        similar = [user for user in sim if check_if_same_type(int(user[0]),etype)]
        similar = pd.DataFrame(similar, columns=['twitter_id', 'similarity']).head(topn)
        
        return pd.merge(similar, sv.entities, on='twitter_id', how='left')

In [ ]:
# Read all users again, now we will use them for testing the recomendation performance

In [ ]:
df_all = df_all[df_all['friends_ids'].notna()]

In [ ]:

sample_size = 20000

df_subset_users = df_all.sample(sample_size)
df_subset_users["friends_ids"] = df_subset_users.progress_apply(parse_friends_ids, axis=1)

# Check similarities

Next we read again the accounts from the excel, after marking manually which accounts to use under the column "use"


In [ ]:
# Use the ExcelFile class to read the Excel file
excel_file = pd.ExcelFile("popular_accounts_1.xlsx")

# Get all sheet names
sheet_names = excel_file.sheet_names
print(sheet_names)

In [ ]:
category = 'TelevisionHost'

In [ ]:
df_rec = pd.read_excel("popular_accounts_1.xlsx", sheet_name='TelevisionHost')
relevant_accounts = df_rec[df_rec['use']==1]

users_to_remove = []

for i, ac in relevant_accounts.iterrows():
    try:
        sv[ac['twitter_user_id']]
    except:
        users_to_remove.append(ac['twitter_user_id'])

        
relevant_accounts = relevant_accounts[~relevant_accounts['twitter_user_id'].isin(users_to_remove)]

print(f"removed {users_to_remove} users")
print(f"remaining {relevant_accounts.shape[0]} users")


### Similarity-Based Recommendation Analysis

The presented code segment evaluates the effectiveness of similarity-based recommendations for social media accounts, specifically analyzing Twitter user behavior. 

#### Experimental Setup
- A subset consisting of **10,000 accounts** is selected from a larger dataset.
- For each user, the algorithm isolates the accounts they follow (`friends_ids`) and identifies a subset after removing previously identified relevant accounts.

#### Methodology
- Embedding vectors from a predefined embedding space (`sv`) are utilized.
- A user's representative vector is computed by averaging embeddings of followed but non-relevant accounts.
- Cosine similarity is calculated between the user's representative vector and each relevant account's vector, resulting in a similarity-ranked list.

#### Evaluation Metrics
- The positions of removed relevant accounts within the similarity-ranked list are noted to quantify recommendation effectiveness.
- These positions are also compared to a popularity-based ranking, which serves as a baseline.

Two key metrics are computed:
- **Reciprocal Rank**: Indicates how highly the removed relevant accounts are ranked.
- **Average Precision (AP)**: Measures precision across ranks, evaluating the accuracy of recommendations.

#### Results Compilation
- A comprehensive report DataFrame is generated, documenting:
  - Individual user analyses
  - Rankings of removed accounts
  - Reciprocal ranks
  - Average precision scores

This structured approach enables an effective comparison between similarity-based recommendations and conventional popularity-driven methods.



In [ ]:
num_of_accounts = 10000
df_subset_users = df_all.head(num_of_accounts)

def calc_similarities(df_subset_users, relevant_accounts):
    df_report = pd.DataFrame()
    
    relevant_account_list = relevant_accounts['twitter_user_id'].values
    
    for i, user in df_subset_users.head(num_of_accounts).iterrows():
        user_list = user['friends_ids']
        user_list_subset = list(set(user_list) - set(relevant_account_list))
        removed_accounts = set(user_list)-set(user_list_subset)
        if removed_accounts == 0:
            continue
        removed_accounts_name = [df_rec[df_rec['twitter_user_id']==x].iloc[0]['twitter_name'] for x in removed_accounts]
        
            
        if len(removed_accounts_name) > 0:
            user_sv = sv.get_average_embeddings(user_list_subset)[0]

            df_similarity_results = pd.DataFrame()

            for account in relevant_account_list:
                try:
                    sim = cosine(sv[int(account)],user_sv)
                except:
                    print(f"Account {account} type:{type(account)} missing")
                    assert ""
                df_similarity_results = pd.concat([df_similarity_results, 
                          pd.DataFrame.from_dict({'account': [account],
                                'similarity':[sim]})])
            #except Exception as e:
                        #    print(e)

            sorted_recommendation = pd.merge(df_similarity_results.sort_values(by='similarity',ascending=True),
                 relevant_accounts[['twitter_user_id','twitter_screen_name','twitter_name']], left_on='account',right_on='twitter_user_id')
            
            popular_recommendation = relevant_accounts.sort_values(by='followers_count', ascending=False).reset_index(drop=True)
            
            removed_accounts_positions = []
            popular_accounts_positions = []
            
            for item in removed_accounts:
                removed_accounts_positions.append(sorted_recommendation[sorted_recommendation['account']==item].index[0]+1)
                popular_accounts_positions.append(relevant_accounts[relevant_accounts['twitter_user_id']==item].index[0]+1)

            print(f"removed accounts: {removed_accounts_name}\n-- reommendation similarity positions: {sorted(removed_accounts_positions)} out of {len(sorted_recommendation)}")
            print(f"-- popular positions: {popular_accounts_positions}")
            
            sv_reciprocal_rank = sum([1/acc for acc in removed_accounts_positions]) / len(removed_accounts_positions)
            pop_reciprocal_rank = sum([1/acc for acc in popular_accounts_positions]) / len(removed_accounts_positions)
            
            # Compute precision at each relevant position
            precisions = []
            for i, rank in enumerate(removed_accounts_positions, start=1):
                precisions.append(i / rank)
            sv_ap = sum(precisions) / len(removed_accounts_positions) if removed_accounts_positions else 0
            
            precisions = []
            for i, rank in enumerate(popular_accounts_positions, start=1):
                precisions.append(i / rank)
            pop_ap = sum(precisions) / len(removed_accounts_positions) if removed_accounts_positions else 0
            
            print(f"num_of_removed_accounts: {len(removed_accounts_positions)}")
            print(f"sv_ap:  {sv_ap:.03f}")
            print(f"pop_ap: {pop_ap:.03f}")
            
            df_report = pd.concat([df_report,
                                  pd.DataFrame.from_dict({
                                      'twitter_user':[user['screen_name']],
                                      'removed_accounts': [removed_accounts_name],
                                      'similarity_rank': [removed_accounts_positions],
                                      'sv_reciprocal_rank': [sv_reciprocal_rank],
                                      'pop_reciprocal_rank': [pop_reciprocal_rank],
                                      'sv_ap': [sv_ap],
                                      'pop_ap': [pop_ap],
                                  })])
            if df_report.shape[0]>=1000:
                break
            
    return df_report

    


In [ ]:
report = calc_similarities(df_subset_users, relevant_accounts)

In [ ]:
report

In [ ]:
sheet_names = ['cars']

In [ ]:
mrr_report = pd.DataFrame()
for category in sheet_names:
    
    # Read the relevant accounts
    df_rec = pd.read_excel("popular_accounts_1.xlsx", sheet_name=category)
    relevant_accounts = df_rec[df_rec['use']==1]

    users_to_remove = []

    for i, ac in relevant_accounts.iterrows():
        try:
            sv[ac['twitter_user_id']]
        except:
            users_to_remove.append(ac['twitter_user_id'])


    relevant_accounts = relevant_accounts[~relevant_accounts['twitter_user_id'].isin(users_to_remove)]

    report = calc_similarities(df_subset_users, relevant_accounts)

    mrr_report = pd.concat([mrr_report,
                report.describe().loc['mean'].to_frame().rename(columns={'mean': f'{category}_mean'}).T
                       ])
mrr_report

In [ ]:
sorted_recommendation[sorted_recommendation['account']==item]

# Sample 20 users per account

In [ ]:
topics = pd.ExcelFile("popular_accounts_1.xlsx").sheet_names
mrr_report = pd.DataFrame()

for topic in topics:
    # Read the relevant accounts
    print(topic)
    df_rec = pd.read_excel("popular_accounts_1.xlsx", sheet_name=topic)
    relevant_accounts = df_rec[df_rec['use']==1]
    mrr_detailed_report = pd.DataFrame()
    
    for i, account in tqdm(relevant_accounts.iterrows(), total = relevant_accounts.shape[0]):
        
        account_to_remove = int(account['twitter_user_id'])
        try:
            users_to_evaluate = df_all[df_all['friends_ids'].apply(lambda x: account_to_remove in x)].sample(n=50, random_state=1000)
        except:
            print(f"{account['twitter_name']} - no followers")
            assert ""
        #users_to_evaluate['friends_ids_subset'] = users_to_evaluate['friends_ids'].apply(lambda x: list(set(x)-set([account_to_remove])))
        
        report = calc_similarities(users_to_evaluate, relevant_accounts)

        mrr_detailed_report = pd.concat([mrr_detailed_report,
                report.describe().loc['mean'].to_frame().rename(columns={'mean': f'{topic}_{account["twitter_name"]}'}).T
                       ])
    mrr_report = pd.concat([mrr_report,
                            mrr_detailed_report.describe().loc['mean'].to_frame().rename(columns={'mean': f'{topic}'}).T
               ])


# Calc Similarities with precision

In [ ]:
num_of_accounts = 10000
df_subset_users = df_all.head(num_of_accounts)

def calc_similarities2(df_subset_users, relevant_accounts):
    df_report = pd.DataFrame()
    
    relevant_account_list = relevant_accounts['twitter_user_id'].values
    
    for i, user in df_subset_users.head(num_of_accounts).iterrows():
        user_list = user['friends_ids']
        user_list_subset = list(set(user_list) - set(relevant_account_list))
        removed_accounts = set(user_list)-set(user_list_subset)
        if removed_accounts == 0:
            continue
        removed_accounts_name = [df_rec[df_rec['twitter_user_id']==x].iloc[0]['twitter_name'] for x in removed_accounts]
        
            
        if len(removed_accounts_name) > 0:
            user_sv = sv.get_average_embeddings(user_list_subset)[0]

            df_similarity_results = pd.DataFrame()

            for account in relevant_account_list:
                try:
                    sim = cosine(sv[int(account)],user_sv)
                except:
                    print(f"Account {account} type:{type(account)} missing")
                    assert ""
                df_similarity_results = pd.concat([df_similarity_results, 
                          pd.DataFrame.from_dict({'account': [account],
                                'similarity':[sim]})])
            #except Exception as e:
                        #    print(e)

            sorted_recommendation = pd.merge(df_similarity_results.sort_values(by='similarity',ascending=True),
                 relevant_accounts[['twitter_user_id','twitter_screen_name','twitter_name']], left_on='account',right_on='twitter_user_id')
            
            popular_recommendation = relevant_accounts.sort_values(by='followers_count', ascending=False).reset_index(drop=True)
            
            removed_accounts_positions = []
            popular_accounts_positions = []
            
            for item in removed_accounts:
                removed_accounts_positions.append(sorted_recommendation[sorted_recommendation['account']==item].index[0]+1)
                popular_accounts_positions.append(relevant_accounts[relevant_accounts['twitter_user_id']==item].index[0]+1)

            #print(f"removed accounts: {removed_accounts_name}\n-- reommendation similarity positions: {sorted(removed_accounts_positions)} out of {len(sorted_recommendation)}")
            #print(f"-- popular positions: {popular_accounts_positions}")
            
            sv_map = sum(np.arange(1, len(removed_accounts_positions)+1) / np.array(sorted(removed_accounts_positions))) / len(removed_accounts_positions)
            pop_map = sum(np.arange(1, len(popular_accounts_positions)+1) / np.array(sorted(popular_accounts_positions))) / len(popular_accounts_positions)
            
            #print(f"num_of_removed_accounts: {len(removed_accounts_positions)}")
            #print(f"sv_ap:  {sv_map:.03f}")
            #print(f"pop_ap: {pop_map:.03f}")
            
            df_report = pd.concat([df_report,
                                  pd.DataFrame.from_dict({
                                      'twitter_user':[user['screen_name']],
                                      'removed_accounts': [removed_accounts_name],
                                      'similarity_rank': [sorted(removed_accounts_positions)],
                                      'popularity_rank': [sorted(popular_accounts_positions)],
                                      'sv_map': [sv_map],
                                      'pop_map': [pop_map],
                                  })])
            if df_report.shape[0]>=1000:
                break
            
    return df_report

#calc_similarities2(users_to_evaluate.head(1), relevant_accounts)


In [ ]:
topics = pd.ExcelFile("popular_accounts_2.xlsx").sheet_names
map_report = pd.DataFrame()
agg_report = pd.DataFrame()
agg_detailed_report = pd.DataFrame()

for topic in topics:
    # Read the relevant accounts
    print(topic)
    df_rec = pd.read_excel("popular_accounts_2.xlsx", sheet_name=topic)
    relevant_accounts = df_rec[df_rec['use']==1]
    map_detailed_report = pd.DataFrame()
    
    for i, account in tqdm(relevant_accounts.iterrows(), total = relevant_accounts.shape[0]):
        
        account_to_remove = int(account['twitter_user_id'])
        try:
            users_to_evaluate = df_all[df_all['friends_ids'].apply(lambda x: account_to_remove in x)].sample(n=50, random_state=1000)
        except:
            print(f"{account['twitter_name']} - no followers")
            assert ""
        #users_to_evaluate['friends_ids_subset'] = users_to_evaluate['friends_ids'].apply(lambda x: list(set(x)-set([account_to_remove])))
        
        report = calc_similarities2(users_to_evaluate, relevant_accounts)
        agg_report = pd.concat([agg_report,report])
        
        map_detailed_report = pd.concat([map_detailed_report,
                report.describe().loc['mean'].to_frame().rename(columns={'mean': f'{topic}_{account["twitter_name"]}'}).T
                       ])
        agg_detailed_report = pd.concat([agg_detailed_report,map_detailed_report])
        
    map_report = pd.concat([map_report,
                            map_detailed_report.describe().loc['mean'].to_frame().rename(columns={'mean': f'{topic}'}).T
               ])


In [ ]:
agg_report

In [ ]:
map_report

In [ ]:
file_path = 'mean_average_percision.xlsx'

with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
    map_report.to_excel(writer, sheet_name='MeanAveragePrecision', index=True)
    agg_report.to_excel(writer, sheet_name="DetailedUsers", index=False)
    agg_detailed_report.to_excel(writer, sheet_name="DetailedAccounts", index=True)

## Add resturants

In [ ]:
rest = pd.read_csv('/workspace/code/sv/restaurants.csv')

In [ ]:
df_rec = pd.merge(rest, df_count, on='twitter_user_id')

In [ ]:
for i, row in df_rec.iterrows():
    try:
        sv[row['twitter_user_id']]
    except:
        print(row['twitter_screen_name'])

In [ ]:
df_rec.columns

In [ ]:
df_rec.sort_values(by='friends_ids', ascending=False, inplace=True)

In [ ]:
df_rec.rename(columns={'friends_ids':'followers_count'}, inplace=True)

In [ ]:
df_rec[['twitter_screen_name','twitter_user_id','twitter_name','wikidata_link', 'wikidata_qid', 'wikidata_label',
       'wikidata_desc','followers_count']]

In [ ]:
relevant_accounts = df_rec.head(20)
map_detailed_report = pd.DataFrame()

for i, account in tqdm(relevant_accounts.iterrows(), total = relevant_accounts.shape[0]):

    account_to_remove = int(account['twitter_user_id'])
    try:
        users_to_evaluate = df_all[df_all['friends_ids'].apply(lambda x: account_to_remove in x)].sample(n=50, random_state=1000)
    except:
        print(f"{account['twitter_name']} - no followers")
        assert ""
    #users_to_evaluate['friends_ids_subset'] = users_to_evaluate['friends_ids'].apply(lambda x: list(set(x)-set([account_to_remove])))

    report = calc_similarities2(users_to_evaluate, relevant_accounts)
    #agg_report = pd.concat([agg_report,report])

    map_detailed_report = pd.concat([map_detailed_report,
            report.describe().loc['mean'].to_frame().rename(columns={'mean': f'{topic}_{account["twitter_name"]}'}).T
                   ])


In [ ]:
map_detailed_report

In [ ]:
report